In [16]:
# Import all necesscary libraries
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [17]:
#Getting a response text from the url
url = 'http://chubbygrub.com'
response = requests.get(url)

In [18]:
#Checking response status
response.status_code

200

In [19]:
#Creating a soup object
soup = BeautifulSoup(response.content, 'lxml')

### Scrape the home page soup for every restaurant

Note: Your best bet is to create a list of dictionaries, one for each restaurant. Each dictionary contains the restaurant's name and slug. The result of your scrape should look something like this:

```python
restaurants = [
    {'name': 'A&W Restaurants', 'slug': 'aw-restaurants'}, 
    {'name': "Applebee's", 'slug': 'applebees'},
    ...
]
```

In [34]:
#Creating a list of dictionaries containg restaurants and their name slugs's
restaurants = []
names = soup.find_all('a', {'class' : 'btn btn-lg btn-primary'})
for i in names:
    restaurant = {}
    restaurant['name'] = i.text
    restaurant['slug'] = i.attrs['href'][13:]
    restaurants.append(restaurant)

In [36]:
len(restaurants)

44

### Step 3: Using the slug, scrape each restaurant's page and create a single list of food dictionaries.

Your list of foods should look something like this:
```python
foods = [
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    {
        'calories': '0',
        'carbs': '0',
        'category': 'Drinks',
        'fat': '0',
        'name': 'A&W® Diet Root Beer',
        'restaurant': 'A&W Restaurants'
    },
    ...
]
```

In [92]:
#Creating an empty list for all food item information
foods = []
#Iterating through all the restaurants found in the previous scrape
for i in restaurants:
    url = 'http://chubbygrub.com/restaurants/'+ i['slug']
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'lxml')
#Finding the main table and all the rows in the table
    table = soup.find('table', {'id': 'items'})
    rows = table.find('tbody').find_all('tr')
#Each row corresponds to a food item and finding all the required info of these items     
    for _ in rows:
        row = _.find_all('td')
        food_item = {}
        food_item['calories'] = row[2].text
        food_item['carbs'] = row[4].text
        food_item['category'] = row[1].text.strip()
        food_item['fat'] = row[3].text
        food_item['name'] = row[0].text
        food_item['restaurant'] = i['name']
        foods.append(food_item)

In [110]:
#Checking the number of food items that were scraped 
print(f'The number of rows in the new DataFrame will be {len(foods)}')

The number of rows in the new DataFrame will be 4977


### Step 4: Create a pandas DataFrame from your list of foods

**Note**: Your DataFrame should have 4,977 rows

In [112]:
#Creating a dataframe called nutri_df from the foods list and checking how it looks
nutri_df  = pd.DataFrame(foods, columns = ['name', 'restaurant', 'category', 'calories', 'carbs', 'fat'])
nutri_df.tail(10)

,name,restaurant,category,calories,carbs,fat
4967,Son of Baconator®,Wendy's,Burgers,670,41,37
4968,Sour Cream and Chive Potato,Wendy's,Sides,320,63,3.5
4969,Spicy Chicken Caesar Salad,Wendy's,Salads,780,40,51
4970,Spicy Chicken Caesar Salad Half-Size,Wendy's,Salads,440,28,27
4971,Spicy Chicken Go Wrap,Wendy's,Wraps,330,30,16
4972,Spicy Chicken Sandwich,Wendy's,Sandwiches,510,55,20
4973,Ultimate Chicken Grill,Wendy's,Sandwiches,370,43,7
4974,Value French Fries,Wendy's,French Fries,220,28,11
4975,Value French Fries,Wendy's,French Fries,220,28,11
4976,Vanilla Frosty Waffle Cone,Wendy's,Shakes,290,53,6


### Step 5: Export to csv

**Note:** Don't export the index column from your DataFrame

In [108]:
#Exporting the dataframe to a csv file
nutri_df.to_csv('Nutrition_Chubby.csv', index = False)